In [1]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
   
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

print("API Key Loaded:", bool(GEMINI_API_KEY))

genai.configure(api_key=GEMINI_API_KEY)

API Key Loaded: True


c:\Users\Asus\Desktop\LangChain-Tutorials\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load PDF file

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./Data folder//Introduction_To_AI.pdf")
documents = loader.load()
  
print(f"Total pages loaded: {len(documents)}")

Total pages loaded: 9


### Split documents into chunks

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  
    chunk_overlap=200
)
docs = text_splitter.split_documents(documents)

print(f"Total chunks created: {len(docs)}")

Total chunks created: 18


### Create embeddings & store in ChromaDB

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

persist_directory = "./chroma_db"

vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory=persist_directory
)

print("Data stored in ChromaDB")

Data stored in ChromaDB


### Query function

In [5]:
def query_with_gemini(question: str, top_k: int = 3):

    results = vectorstore.similarity_search(question, k=top_k)
    if not results:
        return "No data"
    
    context = " ".join([doc.page_content for doc in results])
    
    prompt = f"""You are an AI assistant. Use the following context to answer the question.
    Context: {context}
    Question: {question}"""
    
    model = genai.GenerativeModel()
    response = model.generate_content(prompt)    
    return response.text.strip()

### Ask a test question

In [6]:
if __name__ == "__main__":
    question = "What is Artificial Intelligence?"
    answer = query_with_gemini(question)
    print("Question:", question)
    print("Answer:", answer)

Question: What is Artificial Intelligence?
Answer: Artificial Intelligence (AI) is a computing concept that enables machines to think and solve complex problems like humans do.  AI systems work on a problem, make mistakes, and learn from those mistakes in a self-correcting manner to improve their performance.  This is analogous to humans learning from their errors, such as improving chess strategy by analyzing past losses and adjusting future moves.
